In [ ]:
"""
Plot the hydrofracturing limit
"""

In [ ]:
import numpy as np
import xarray as xr

In [ ]:
inputpath_raw = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/'

In [ ]:
home_path = '/bettik/burgardc/'
inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/'

In [ ]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

In [ ]:
hf_data = np.load(inputpath_raw+'runoff_dates_2300_100.0.npz')

In [ ]:
scen_list = []
for scen in ['ssp126','ssp585']:
    pctl_list = []
    for pctl in [0,10,33,50,66,90,100]:
        varname = 'year_bas_'+scen+'_pct'+str(pctl)
        arr = xr.DataArray(data=hf_data[varname], dims=['Nisf']).assign_coords({'Nisf': file_isf.Nisf})
        pctl_list.append(arr.assign_coords({'pctl': pctl}))
    scen_list.append(xr.concat(pctl_list, dim='pctl').assign_coords({'scenario': scen}))
xr_hf = xr.concat(scen_list, dim='scenario')

In [ ]:
xr_hf.to_dataset(name='hydrofrac_limit').to_netcdf(inputpath_data + 'hydrofracturing_limits.nc')

In [ ]:
lst = hf_data.files
for item in lst:
    print(item)
    print(hf_data[item])